In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics, manifold
from tqdm import tqdm
import gensim.downloader as gensim_api
import transformers
import os

In [2]:
df=pd.read_csv('output1.csv',nrows=4000)
print(df["clean"])

0        rt mleew boy dats cold tyga dwn bad for cuffi...
1        rt urkindofbrand dawg rt sbaby life you ever ...
2        rt c g anderson viva based she look like a tr...
3        rt shenikaroberts the shit you hear about me ...
4        t madison x the shit just blows me claim you ...
                              ...                        
3995            msoltan koredeb the witches took them hoe
3996     msoltan matt simonson koredeb nvm thought we ...
3997     mtbornnard it s got good mojo for sure paybac...
3998     m a larson stevestreza you re saying zebras a...
3999                              m carbonaro nice pussy 
Name: clean, Length: 4000, dtype: object


In [3]:
nlp = gensim_api.load("glove-wiki-gigaword-300")

In [4]:
nlp.most_similar(["nigger"], topn=3)

[('slur', 0.5023007392883301),
 ('niggers', 0.5003144145011902),
 ('faggot', 0.4708784222602844)]

In [5]:
def get_similar_words(lst_words, top, nlp):
    lst_out = lst_words
    for tupla in nlp.most_similar(lst_words, topn=top):
        lst_out.append(tupla[0])
    return list(set(lst_out))

In [6]:
def similarWords(words, top, nlp):
    out = words
    for tupl in nlp.most_similar(words, topn=top):
        out.append(tupl[0])
    return list(set(out))

In [7]:
clusters={}
clusters['HATE']=get_similar_words(['hate'], top=30, nlp=nlp)

In [8]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('al', do_lower_case=True)
nlp = transformers.TFDistilBertModel.from_pretrained('al')

Some layers from the model checkpoint at al were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at al.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [9]:
def utils_bert_embedding(txt, tokenizer, nlp):
    '''
    Word embedding with Bert (equivalent to nlp["word"]).
    :parameter
        :param txt: string 
        :param tokenizer: transformers tokenizer
        :param nlp: transformers bert
    :return
        tensor sentences x words x vector (1x3x768) 
    '''
    # tokenize sentence to tokens (integers)
    idx = tokenizer.encode(txt)
    # convert to array of shape (1, num_words+2) - EOS and CLS added
    idx = np.array(idx)[None,:]
    # generate embeddings for each token - output is a tuple
    embedding = nlp(idx)
    # select first member of the tuple, remove first dimension which is 1 to get (num_words,embedding size 712)
    # exclude CLS and EOS tokens
    X = np.array(embedding[0][0][1:-1])
    return X

In [10]:
%%time
## create list of news vector
lst_mean_vecs = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) for txt in tqdm(df["clean"])]

100%|██████████████████████████████████████████████████████████████████████████████| 4000/4000 [08:40<00:00,  7.68it/s]

Wall time: 8min 40s


In [11]:
X = np.array(lst_mean_vecs)
X.shape

(4000, 768)

In [12]:
np.savetxt("mean_vec.csv", X, delimiter="/")

In [13]:
print(X)

[[-2.56808370e-01  2.32406840e-01  1.69656605e-01 ... -8.87778848e-02
   1.40191754e-02  1.45572618e-01]
 [ 8.45624730e-02  3.25855941e-01  4.16700900e-01 ... -1.73627079e-01
   2.83790618e-01 -5.14129251e-02]
 [ 2.25455880e-01 -2.60253102e-01  3.21632028e-01 ... -3.30394179e-01
   1.92198440e-01  1.67761743e-01]
 ...
 [ 1.43120185e-01  1.55305579e-01  3.58505338e-01 ... -1.13864124e-01
   1.88397214e-01 -3.04630812e-04]
 [ 9.60611254e-02  8.89073536e-02  2.41476282e-01 ...  4.43981290e-02
   1.34531409e-01  1.73412785e-01]
 [ 8.62215906e-02 -2.67725110e-01  4.67932045e-01 ...  6.54157391e-03
  -4.68252897e-02  1.25600457e-01]]


In [14]:
from numpy import genfromtxt
my_data = genfromtxt('mean_vec.csv', delimiter='/')
print(my_data)

[[-2.56808370e-01  2.32406840e-01  1.69656605e-01 ... -8.87778848e-02
   1.40191754e-02  1.45572618e-01]
 [ 8.45624730e-02  3.25855941e-01  4.16700900e-01 ... -1.73627079e-01
   2.83790618e-01 -5.14129251e-02]
 [ 2.25455880e-01 -2.60253102e-01  3.21632028e-01 ... -3.30394179e-01
   1.92198440e-01  1.67761743e-01]
 ...
 [ 1.43120185e-01  1.55305579e-01  3.58505338e-01 ... -1.13864124e-01
   1.88397214e-01 -3.04630812e-04]
 [ 9.60611254e-02  8.89073536e-02  2.41476282e-01 ...  4.43981290e-02
   1.34531409e-01  1.73412785e-01]
 [ 8.62215906e-02 -2.67725110e-01  4.67932045e-01 ...  6.54157391e-03
  -4.68252897e-02  1.25600457e-01]]


In [15]:
dic_y = {k:utils_bert_embedding(v, tokenizer, nlp).mean(0) for k,v in tqdm(clusters.items())}

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.34it/s]


In [19]:
print(dic_y.values())

dict_values([array([ 1.71892285e-01,  2.28143185e-01,  1.99167445e-01, -3.20793569e-01,
       -9.15243626e-02, -2.25772694e-01,  5.36811233e-01,  3.09728801e-01,
       -3.62344757e-02, -3.10321510e-01,  1.35611713e-01, -2.56128579e-01,
       -2.17093483e-01, -4.28963173e-03,  4.75885645e-02,  2.29074992e-02,
        7.20219985e-02,  1.89861745e-01, -2.08709314e-01,  5.72076857e-01,
        2.13854715e-01,  2.02479765e-01, -9.12104324e-02, -4.25510146e-02,
        2.49153599e-01,  2.96713412e-01,  4.28119540e-01, -2.54093796e-01,
       -1.40116721e-01, -2.02707723e-01,  5.92797458e-01,  1.53673932e-01,
        5.81915304e-02,  1.05630063e-01, -1.16783835e-01,  7.74745047e-02,
        7.60922208e-02,  1.96703285e-01,  1.31298706e-01,  2.53739685e-01,
       -4.27306563e-01, -3.47440898e-01,  6.37586266e-02,  2.86770582e-01,
       -3.98509264e-01, -4.26786333e-01, -3.38047117e-01,  1.07258148e-01,
        8.57259408e-02, -2.85049736e-01, -5.46136573e-02,  1.86780825e-01,
       -1.56

In [22]:
similarities = np.array(
            [metrics.pairwise.cosine_similarity(X, [dic_y['HATE']]).T.tolist()[0]]
            ).T

In [23]:
print(similarities)

[[0.61511302]
 [0.73330647]
 [0.68261987]
 ...
 [0.65181994]
 [0.6799947 ]
 [0.57162768]]


In [ ]:
labels=list(dic_y.keys())
similarities[0]=similarities[0]/sum(similarities[0])